In [1]:
from bids import BIDSLayout
from os.path import join, dirname
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob



In [2]:
#get bids layouts

bids_list = [{'bids_dir': 'cfmm-bids/Khan/SNSX_7T_fresh/bids', 'project': 'SNSX7T'},
             {'bids_dir': 'cfmm-bids/Khan/EpLinkPhase3_7T/bids', 'project': 'EpLink7T'}, 
              {'bids_dir': 'bids_addiction', 'project': 'addiction'},
               {'bids_dir': 'bids_topsy', 'project': 'topsy'}]

layout_list = [ BIDSLayout(bids_list[i]['bids_dir'],derivatives=False,validate=False,index_metadata=True,database_path='.db_cache.{}'.format(bids_list[i]['project'])) for i in range(len(bids_list))]


In [6]:

for i,layout in enumerate(layout_list):

    project = bids_list[i]['project']
    
    for bids_img in layout.get(suffix='MP2RAGE',extension='nii.gz',acquisition='UNI'):

        subject = bids_img.get_entities()['subject']

        #load scans.tsv for getting acquisition date, since it is not in the metadata
        scans_tsv = glob(join(dirname(bids_img.path),'..',f'sub-{subject}_*scans.tsv'))[0] 
        scans_df = pd.read_table(scans_tsv)

         #get date from acq_time string in first file
        acq_date = scans_df['acq_time'][0][:10]


        img = bids_img.get_image()
        vol = img.get_fdata()

        #affine from RAS space to voxel coords, use this to get voxel coords of isocentre
        zero = np.zeros((4,1))
        zero[-1] = 1
        isocenter = np.linalg.inv(img.affine) @ zero
        isocenter = isocenter.astype('int')

        #get inv2 image to estimate head size
        mp2rage_imglist = layout.get(suffix='MP2RAGE',extension='nii.gz',subject=subject)
        inv2 = [i for i in mp2rage_imglist if 'inv-2' in i.filename] 
        inv2_img = inv2.pop().get_image()


        vol = inv2_img.get_fdata()

        plt.figure(figsize=(15,6))


        #plot sagittal slice with isocenter
        sag_slice = np.rot90(vol[isocenter[0],:,:].squeeze())
        plt.subplot(1,4,1)
        plt.imshow(sag_slice)
        plt.hlines(vol.shape[2]-isocenter[2],0,vol.shape[1],colors='yellow')
        plt.vlines(isocenter[1],0,vol.shape[2],colors='yellow')
        plt.axis('off');

        plt.title(f'subject: {subject}, acq_date: {acq_date}')

        #plot coronal slice with isocenter

        cor_slice = np.rot90(vol[:,isocenter[1],:].squeeze())
        plt.subplot(1,4,2)
        plt.imshow(cor_slice)
        plt.hlines(vol.shape[2]-isocenter[2],0,vol.shape[0],colors='yellow')
        plt.vlines(isocenter[0],0,vol.shape[2],colors='yellow')
        plt.axis('off');

        proj_labels = ('sagittal','coronal')
        for ax in (0,1):
            proj = np.rot90(np.max(vol,ax))
            plt.subplot(1,4,ax+3)
            plt.imshow(proj)

            proj_label = proj_labels[ax]
            plt.title(f'{proj_label}')
            plt.axis('off');

        plt.savefig(f'{acq_date}_project-{project}_sub-{subject}.png')
        plt.close()

